In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import *
from tensorflow.keras.applications import ResNet50

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -r /content/drive/MyDrive/archive.tar /content/

In [ ]:
!tar -xf /content/archive.tar -C /content/

In [ ]:
train_dir = "/content/data/fruits-360_100x100/fruits-360/Training"
test_dir = "/content/data/fruits-360_100x100/fruits-360/Test"

In [ ]:
train_tfms = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.Lambda(
        tf.keras.applications.resnet50.preprocess_input
    )
])


val_tfms = tf.keras.Sequential([
    tf.keras.layers.Lambda(
        tf.keras.applications.resnet50.preprocess_input
    )
])


In [ ]:
!cp -r /content/drive/MyDrive/Fruits-360_/model/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5 /content/

In [ ]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32
)

num_classes = len(train_ds.class_names)
print(f"Number of classes: {num_classes}")

Found 122118 files belonging to 232 classes.
Number of classes: 232


In [ ]:
model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 232)            │       119,016 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,764,008 (94.47 MB)

 Trainable params: 1,172,200 (4.47 MB)

 Non-trainable params: 23,591,808 (90.00 MB)

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32
)

train_ds = train_ds.map(
    lambda x, y: (train_tfms(x, training=True), y),
    num_parallel_calls=tf.data.AUTOTUNE
)

val_ds = test_ds.map(
    lambda x, y: (val_tfms(x, training=False), y),
    num_parallel_calls=tf.data.AUTOTUNE
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

Found 40715 files belonging to 232 classes.
Epoch 1/5
3817/3817 ━━━━━━━━━━━━━━━━━━━━ 1037s 268ms/step - accuracy: 0.7593 - loss: 1.1033 - val_accuracy: 0.9808 - val_loss: 0.0585
Epoch 2/5
3817/3817 ━━━━━━━━━━━━━━━━━━━━ 1022s 268ms/step - accuracy: 0.9796 - loss: 0.0642 - val_accuracy: 0.9885 - val_loss: 0.0434
Epoch 3/5
3817/3817 ━━━━━━━━━━━━━━━━━━━━ 1047s 274ms/step - accuracy: 0.9834 - loss: 0.0478 - val_accuracy: 0.9899 - val_loss: 0.0361
Epoch 4/5
3817/3817 ━━━━━━━━━━━━━━━━━━━━ 1024s 268ms/step - accuracy: 0.9877 - loss: 0.0408 - val_accuracy: 0.9924 - val_loss: 0.0315
Epoch 5/5
3817/3817 ━━━━━━━━━━━━━━━━━━━━ 1054s 276ms/step - accuracy: 0.9894 - loss: 0.0347 - val_accuracy: 0.9902 - val_loss: 0.0386


In [ ]:
test_ds_eval = test_ds.map(lambda x, y: (val_tfms(x, training=False), y))

model.evaluate(test_ds_eval)


1273/1273 ━━━━━━━━━━━━━━━━━━━━ 119s 93ms/step - accuracy: 0.9932 - loss: 0.0231


[0.02125268429517746, 0.9939088821411133]

In [ ]:
model.save('/content/classifier.keras')

In [ ]:
!cp -r /content/classifier.keras /content/drive/MyDrive/Fruits-360_

In [ ]:
loaded_model = tf.keras.models.load_model('/content/classifier.keras')
print("Model loaded successfully!")

Model loaded successfully!


In [ ]:
def preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    processed_img = tf.keras.applications.resnet50.preprocess_input(img_array)
    return processed_img

print("Preprocessing function defined.")

Preprocessing function defined.


In [ ]:
def predict_image(image_path):
    # Preprocess the image
    processed_img = preprocess_image(image_path)

    # Make prediction
    predictions = loaded_model.predict(processed_img)

    # Get the predicted class index and its probability
    predicted_class_index = np.argmax(predictions[0])
    predicted_probability = np.max(predictions[0])

    # Get class names from train_ds (assuming train_ds is available globally with class_names)
    # We need to make sure train_ds.class_names is accessible here. It was defined in cell ce6b596b.
    class_names = train_ds.class_names

    # Get the predicted class label
    predicted_class_label = class_names[predicted_class_index]

    return predicted_class_label, predicted_probability

print("Predict image function defined.")

Predict image function defined.


In [ ]:
import os

class_names_list = sorted(os.listdir(train_dir))
print("Class names retrieved successfully:")
print(class_names_list)

Class names retrieved successfully:
['Apple 10', 'Apple 11', 'Apple 12', 'Apple 13', 'Apple 14', 'Apple 17', 'Apple 18', 'Apple 19', 'Apple 5', 'Apple 6', 'Apple 7', 'Apple 8', 'Apple 9', 'Apple Braeburn 1', 'Apple Core 1', 'Apple Crimson Snow 1', 'Apple Golden 1', 'Apple Golden 2', 'Apple Golden 3', 'Apple Granny Smith 1', 'Apple Pink Lady 1', 'Apple Red 1', 'Apple Red 2', 'Apple Red 3', 'Apple Red Delicious 1', 'Apple Red Yellow 1', 'Apple Red Yellow 2', 'Apple Rotten 1', 'Apple hit 1', 'Apple worm 1', 'Apricot 1', 'Avocado 1', 'Avocado Black 1', 'Avocado Black 2', 'Avocado Green 1', 'Avocado ripe 1', 'Banana 1', 'Banana 3', 'Banana 4', 'Banana Lady Finger 1', 'Banana Red 1', 'Beans 1', 'Beetroot 1', 'Blackberrie 1', 'Blackberrie 2', 'Blackberrie half rippen 1', 'Blackberrie not rippen 1', 'Blueberry 1', 'Cabbage red 1', 'Cabbage white 1', 'Cactus fruit 1', 'Cactus fruit green 1', 'Cactus fruit red 1', 'Caju seed 1', 'Cantaloupe 1', 'Cantaloupe 2', 'Carambula 1', 'Carrot 1', 'Caulifl

In [ ]:
import os
import random

# Get a list of all class directories in the test_dir
all_test_classes = os.listdir(test_dir)

# Randomly pick one class
random_class = random.choice(all_test_classes)

# Get the path to that random class directory
random_class_dir = os.path.join(test_dir, random_class)

# Get a list of all images in that random class directory
images_in_class = os.listdir(random_class_dir)

# Randomly pick one image from that class
example_image_filename = random.choice(images_in_class)

# Construct the full path to the example image
example_image_path = os.path.join(random_class_dir, example_image_filename)

print(f"Selected example image: {example_image_path}")

Selected example image: /content/data/fruits-360_100x100/fruits-360/Test/Watermelon 1/296_100.jpg


In [ ]:
predicted_label, predicted_prob = predict_image(example_image_path)

print(f"Predicted label: {predicted_label}")
print(f"Predicted probability: {predicted_prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Predicted label: Watermelon 1
Predicted probability: 1.0000


In [ ]:
from google.colab import files
import io

# Upload a file
uploaded = files.upload()

# Get the name of the uploaded file
if uploaded:
    uploaded_filename = list(uploaded.keys())[0]
    uploaded_file_path = f"/content/{uploaded_filename}"
    print(f"Uploaded file: {uploaded_filename}")

    # Make a prediction with the uploaded file
    predicted_label_uploaded, predicted_prob_uploaded = predict_image(uploaded_file_path)

    print(f"\nPredicted label for uploaded photo: {predicted_label_uploaded}")
    print(f"Predicted probability for uploaded photo: {predicted_prob_uploaded:.4f}")
else:
    print("No file was uploaded.")

Saving istockphoto-184276818-612x612.jpg to istockphoto-184276818-612x612.jpg
Uploaded file: istockphoto-184276818-612x612.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step

Predicted label for uploaded photo: Pear Red 1
Predicted probability for uploaded photo: 0.9999
